# Libraries

In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, confusion_matrix

# Setting up device

In [9]:
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Downloading Train and Test MNIST set

In [10]:
train_ds = torchvision.datasets.MNIST(root="./data",train=True,transform=transforms.ToTensor(),download=True)
test_ds = torchvision.datasets.MNIST(root="./data",train=False,transform=transforms.ToTensor())

# Data Loader

In [11]:
train_dl = torch.utils.data.DataLoader(dataset=train_ds, batch_size = 100, shuffle=True)
test_dl = torch.utils.data.DataLoader(dataset=test_ds, batch_size = 100, shuffle=False)

# MLP model class

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200, 50)
        self.fc3 = nn.Linear(50, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Assigning model to class

In [15]:
model = Net().to(device)

# Criterion (Loss) and Optimizer  

In [16]:
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)

# Training

In [17]:
num_epochs = 3

epochs = tqdm(range(num_epochs))

for epoch in epochs:
    train_loss = []
    model.train()
    for batch in (train_dl):
        optim.zero_grad()
        images = batch[0].reshape(-1, 28*28).to(device)
        label = batch[1].to(device)
        # Forward pass
        output = model(images)
        loss = torch.nn.CrossEntropyLoss()(output.squeeze(-1),label)
        # Backward-pass and optimize
        loss.backward()
        optim.step()

    loss_now = np.mean(train_loss)
    epochs.set_postfix({"loss":loss_now})

  0%|          | 0/3 [00:00<?, ?it/s]

/opt/homebrew/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


# Test

In [19]:
predlist = torch.zeros(0,dtype=torch.long,device="cpu")
lbllist = torch.zeros(0,dtype=torch.long, device="cpu")

with torch.no_grad():
    for ims, labels in test_dl:
        ims = ims.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(ims)
        _, predicted = torch.max(outputs.data, 1)
        # Append batch prediction results
        predlist = torch.cat([predlist,predicted.view(-1).cpu()])
        lbllist = torch.cat([lbllist,labels.view(-1).cpu()])

# Confusion Matrix
conf_mat = confusion_matrix(np.asarray(lbllist), np.asarray(predlist))
print(conf_mat)

# Per-class accuracy

class_accuracy = 100 * conf_mat.diagonal()/conf_mat.sum(1)
print(class_accuracy)

# Report

print(classification_report(np.asarray(lbllist), np.asarray(predlist), digits=3))

[[ 968    0    0    2    1    1    5    1    1    1]
 [   0 1122    4    0    0    1    5    0    3    0]
 [   4    1 1007    0    3    1    1    6    8    1]
 [   0    0    8  954    0   24    0    6    6   12]
 [   0    0    5    0  955    0    6    1    2   13]
 [   2    1    0    3    2  868    8    0    5    3]
 [   7    2    0    0    4    7  935    0    3    0]
 [   1    6   17    1    0    1    0  984    4   14]
 [   5    0    4    4    3    3    9    3  938    5]
 [   5    4    0    4   13    1    0    2    2  978]]
[98.7755102  98.85462555 97.57751938 94.45544554 97.25050916 97.30941704
 97.59916493 95.71984436 96.30390144 96.92765114]
              precision    recall  f1-score   support

           0      0.976     0.988     0.982       980
           1      0.988     0.989     0.988      1135
           2      0.964     0.976     0.970      1032
           3      0.986     0.945     0.965      1010
           4      0.973     0.973     0.973       982
           5      0.9